In [1]:
from numpy.core.fromnumeric import amin
import requests
from bs4 import BeautifulSoup
from datetime import date
import re 
import numpy as np 
import pandas as pd 
from IPython.display import clear_output
from time import sleep
from itertools import islice
import time

In [2]:
def get_index(iterable, value, n):
    matches = (idx for idx, val in enumerate(iterable) if value in val)
    return next(islice(matches, n-1, n), None)

In [3]:
def wallmine_data(pages):
    Symbol = []
    Book_Share = []
    Cash_Share = []
    Dividend_Yield = []
    Beta = []
    Company = []
    Current_Ratio = []
    Debt_Equity = []
    P_E = []
    Industry = []
    Price_Book = []
    Sector = []
    ROA = []
    ROE = []
    Payment_Date = []
    EPS_QQ = []
    PEG = []
    Payout_Ratio = []
    Price_Sales = []
    Profit_Margin = []
    Choose_column = []

    # URL de inicio de sesión
    login_url = 'https://wallmine.com/users/sign-in'

    # Datos de inicio de sesión
    login_data = {
        'utf8': '✓',
        'authenticity_token': '',
        'user[email]': '****',
        'user[password]': '****',
        'user[remember_me]': '0',
        'commit': 'Sign in'
    }

    # Cabeceras
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    
    with requests.Session() as s:
        # Obtener el token de autenticidad
        response = s.get(login_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        token = soup.find('input', {'name': 'authenticity_token'}).get('value')

        # Agregar el token de autenticidad a los datos de inicio de sesión
        login_data['authenticity_token'] = token

        # Iniciar sesión
        response = s.post(login_url, data=login_data, headers=headers)

        for i in pages:
            main_url = 'https://wallmine.com/screener?bcm=relative_sma_50_days&d=d&f=bv%2Ccps%2Cdy%2Cb%2Cn%2Ccr%2Cch%2Cd2e%2Ceb%2Cpe%2Ci%2Cp2b%2Cse%2Croa%2Croe%2Cm%2Cp%2Cdd%2Cqeg%2Cpeg%2Cpr%2Cp2s%2Cpm&o=m&page='+str(i)+'&r=cu'

            main_response = s.get(main_url, headers=headers)
            main_soup = BeautifulSoup(main_response.text,'html.parser')

            tabla = main_soup.find('table',class_='table table-striped table-sm table--no-margin screener-results')       
            header = tabla.find('tr')
            header2 = header.find_all('a',class_='js-tooltip')
            cols = []

            for i in header2:
                col = i.text
                cols.append(col)

            rows = tabla.find_all('tr',class_='js-clickable-row clickable-row')
            rows1 = []
            for i in rows:
                test = i.find_all('td')
                temp = []
                for j in test:
                    temp.append(j.text)
                rows1.append(temp)

            for i in rows1:
                Symbol.append(i[0])
                Book_Share.append(i[1])
                Cash_Share.append(i[2])
                Dividend_Yield.append(i[3])
                Beta.append(i[4])
                Company.append(i[5])
                Current_Ratio.append(i[6])
                Debt_Equity.append(i[8])
                P_E.append(i[10])
                Industry.append(i[11])
                Price_Book.append(i[12])
                Sector.append(i[13])
                ROA.append(i[14])
                ROE.append(i[15])
                Payment_Date.append(i[18])
                EPS_QQ.append(i[19])
                PEG.append(i[20])
                Payout_Ratio.append(i[21])
                Price_Sales.append(i[22])
                Profit_Margin.append(i[23])
                
    df = pd.DataFrame({'Symbol':Symbol,
                      'Company':Company,
                      'Sector':Sector,                         
                      'Industry':Industry,       
                      'ROA':ROA,
                      'ROE':ROE,
                      'Current_Ratio':Current_Ratio,
                      'Beta':Beta,           
                      'Debt_Equity':Debt_Equity,      
                      'P_E':P_E,          
                      'EPS_QQ':EPS_QQ,
                      'PEG':PEG,                       
                      'Book_Share':Book_Share,
                      'Cash_Share':Cash_Share,
                      'Dividend_Yield':Dividend_Yield,
                      'Price_Book':Price_Book,
                      'Payment_Date':Payment_Date,
                      'Payout_Ratio':Payout_Ratio,
                      'Price_Sales':Price_Sales,
                      'Profit_Margin':Profit_Margin
                      })
    
    return df

In [4]:
secuencia = [i for i in range(1,101)]
secuencia2 = [i for i in range(101,201)]
secuencia3 = [i for i in range(201,301)]
secuencia4 = [i for i in range(301,401)]
secuencia5 = [i for i in range(401,501)]
secuencia6 = [i for i in range(501, 551)]

df1 = wallmine_data(secuencia)

In [5]:
df2 = wallmine_data(secuencia2)

In [6]:
df3 = wallmine_data(secuencia3)

In [7]:
df4 = wallmine_data(secuencia4)

In [8]:
df5 = wallmine_data(secuencia5)

In [9]:
df6 = wallmine_data(secuencia6)

In [14]:
df = pd.concat([df1, df2, df3, df4, df5, df6])

df = df.applymap(lambda x: np.nan if x == '-' else x)

cols_to_modify = ['Book_Share', 'Cash_Share', 'Dividend_Yield', 'Beta', 'Current_Ratio', 'Debt_Equity',
                 'P_E', 'Price_Book', 'ROA', 'ROE', 'EPS_QQ', 'PEG',
                 'Payout_Ratio', 'Price_Sales', 'Profit_Margin']

for col in cols_to_modify:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace('%', '')
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(df.dtypes)
df.head()

Symbol             object
Company            object
Sector             object
Industry           object
ROA               float64
ROE               float64
Current_Ratio     float64
Beta              float64
Debt_Equity       float64
P_E               float64
EPS_QQ            float64
PEG               float64
Book_Share        float64
Cash_Share        float64
Dividend_Yield    float64
Price_Book        float64
Payment_Date       object
Payout_Ratio      float64
Price_Sales       float64
Profit_Margin     float64
dtype: object


,Symbol,Company,Sector,Industry,ROA,ROE,Current_Ratio,Beta,Debt_Equity,P_E,EPS_QQ,PEG,Book_Share,Cash_Share,Dividend_Yield,Price_Book,Payment_Date,Payout_Ratio,Price_Sales,Profit_Margin
0,APC,Apple Inc.,Consumer Discretionary,Consumer Electronics,28.10,160.90,0.88,1.22,5.96,20.79,NaN,2.80,3.81,3.63,0.59,39.10,N/A,12.86,5.02,25.31
1,AAPL,Apple Inc,Consumer Discretionary,Consumer Electronics,27.46,163.45,0.94,1.30,5.96,28.29,4.03,2.75,4.30,4.10,0.64,38.93,N/A,18.36,5.09,25.31
2,MSF,Microsoft Corporation,Technology,Internet Services & Infrastructure,19.80,42.10,1.84,0.93,1.19,34.84,NaN,2.29,26.49,16.66,0.85,11.51,N/A,29.70,9.44,36.69
3,MSFT,Microsoft Corporation,Technology,Internet Services & Infrastructure,18.79,38.46,1.91,1.26,1.19,32.91,3.52,2.17,27.13,17.06,0.85,11.22,8 Jun 2023,28.11,9.00,36.69
4,4338,Microsoft Corporation,Technology,Internet Services & Infrastructure,18.79,38.46,1.91,0.96,1.19,25.99,NaN,1.81,148.56,93.44,0.16,10.77,N/A,4.22,8.64,36.69


In [15]:
df.to_excel('C:/Users/Acer Nitro 5/Documents/Python Scripts/Wallmine.xlsx', index=False)